# 0. Import Dependencies

In [75]:
import os
import cv2
import random
import numpy as np
import skimage as ski

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# 1. Prepare Samples

In [66]:
positive_samples_dirs = ["../../datasets/faces/natural_images/person/", "../../datasets/faces/images/"]
negative_samples_dirs = [os.path.join("../../datasets/faces/natural_images/", folder) for folder in os.listdir("../../datasets/faces/natural_images/") if folder != "person"]

positive_samples = []
negative_samples = []

for dir in negative_samples_dirs: 
    for image in os.listdir(dir):
        negative_samples.append(os.path.join(dir, image))

for dir in positive_samples_dirs:
    for image in os.listdir(dir):
        positive_samples.append(os.path.join(dir, image))

# 2. Feature Extraction

In [67]:
feature_vector = []
label_vector = []

win_size = (128, 128) 
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
num_bins = 9

HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

for image in positive_samples:

    img = cv2.resize(cv2.imread(image, 0), win_size)

    hog = HOG.compute(img).flatten()
    hog /= hog.max()

    feature_vector.append(hog)
    label_vector.append(1)

for image in negative_samples:

    img = cv2.resize(cv2.imread(image, 0), win_size)

    hog = HOG.compute(img).flatten()
    hog /= hog.max()

    feature_vector.append(hog)
    label_vector.append(0)


# 3. Train the model

In [68]:
x_train, x_test, y_train, y_test = train_test_split(feature_vector, label_vector, test_size=0.2, random_state=42)

face_classifer = LinearSVC().fit(x_train, y_train)

score = face_classifer.score(x_test, y_test)
print(f"Accuracy: {score}")

c:\Tools\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.885227896760022


c:\Tools\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# 4. Real-Time Testing

In [80]:
def extract_features(Image) -> list[list]:

    gray_frame = cv2.resize(cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY), (128, 128))

    hog = HOG.compute(gray_frame).flatten()
    
    return [hog / hog.max()]


camera = cv2.VideoCapture(0)

while camera.isOpened():

    ret, frame = camera.read()

    features = extract_features(frame)

    prediction = face_classifer.predict(features)

    if prediction == 1:
        # Draw a green rectangle if a face is detected
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 3)
    else:
        # Draw a red rectangle if no face is detected
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 3)

    # Display the frame with the detection result
    cv2.imshow('Face Detection', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close the OpenCV windows
camera.release()
cv2.destroyAllWindows()


# 5. Save the model

In [78]:
import joblib

joblib.dump(face_classifer, "Face_Classifying_Model_Trial_I.pkl")

['Face_Classifying_Model_Trial_I.pkl']